In [ ]:
"""
서울시 상권분석서비스 - 상권영역 정보 API 호출
- OA-15560
- 서비스명: TbgisTrdarRelm
- 행정동별 상권유형 개수 집계
+ EDA (탐색적 데이터 분석) 포함
"""

import requests
import pandas as pd
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import sys

# ============================================================
# Jupyter 출력 설정 (잘림 방지)
# ============================================================
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows
# plt.rcParams['font.family'] = 'AppleGothic'  # Mac
plt.rcParams['axes.unicode_minus'] = False

# ============================================================
# 설정
# ============================================================
API_KEY = "4e785a7a5168666f32375a4d787355"  # 매출 API 키 사용 (동일 키로 가능)
SERVICE_NAME = "TbgisTrdarRelm"
BASE_URL = f"http://openapi.seoul.go.kr:8088/{API_KEY}/json/{SERVICE_NAME}"

print(f"서비스명: {SERVICE_NAME}")
print(f"API URL: {BASE_URL}")
sys.stdout.flush()


# ============================================================
# API 호출 함수
# ============================================================
def fetch_all_data():
    """전체 상권영역 데이터 수집"""
    all_data = []
    start = 1
    batch_size = 1000
    max_retries = 3
    max_iterations = 100
    iteration = 0
    
    print(f"\n{'='*60}")
    print("서울시 상권분석 API - 상권영역 정보 데이터 수집 시작")
    print(f"{'='*60}")
    sys.stdout.flush()
    
    while iteration < max_iterations:
        iteration += 1
        end = start + batch_size - 1
        url = f"{BASE_URL}/{start}/{end}"
        
        for retry in range(max_retries):
            try:
                response = requests.get(url, timeout=60)
                response.raise_for_status()
                result = response.json()
                
                if SERVICE_NAME not in result:
                    if 'RESULT' in result:
                        error_msg = result.get('RESULT', {})
                        print(f"API 에러: {error_msg}")
                        if 'INFO-200' in str(error_msg):
                            print("데이터 수집 완료 (더 이상 데이터 없음)")
                            return all_data
                    return all_data
                
                code = result[SERVICE_NAME]['RESULT']['CODE']
                if code != 'INFO-000':
                    print(f"API 응답 코드: {code}")
                    if code == 'INFO-200':
                        return all_data
                    break
                
                rows = result[SERVICE_NAME].get('row', [])
                if not rows:
                    print("더 이상 데이터가 없습니다.")
                    return all_data
                
                all_data.extend(rows)
                total_count = result[SERVICE_NAME]['list_total_count']
                
                print(f"수집 중: {start:,}~{end:,} / 전체 {total_count:,}건 (현재 {len(all_data):,}건)")
                sys.stdout.flush()
                
                if len(all_data) >= total_count:
                    print(f"\n✅ 전체 데이터 수집 완료: {len(all_data):,}건")
                    return all_data
                
                start += batch_size
                time.sleep(0.3)
                break
                
            except requests.exceptions.Timeout:
                print(f"⏰ 타임아웃 (시도 {retry+1}/{max_retries}): {start}~{end}")
                sys.stdout.flush()
                time.sleep(2)
                continue
                
            except requests.exceptions.RequestException as e:
                print(f"❌ 요청 에러 (시도 {retry+1}/{max_retries}): {e}")
                sys.stdout.flush()
                time.sleep(2)
                continue
                
            except Exception as e:
                print(f"❌ 처리 에러: {e}")
                sys.stdout.flush()
                return all_data
        
        else:
            print(f"❌ {start}~{end} 구간 수집 실패, 다음으로 진행")
            start += batch_size
            continue
    
    print(f"⚠️ 최대 반복 횟수 도달 ({max_iterations}회)")
    return all_data


# ============================================================
# 컬럼명 한글 변환
# ============================================================
COLUMN_MAPPING = {
    'TRDAR_SE_CD': '상권구분_코드',
    'TRDAR_SE_CD_NM': '상권구분_명',
    'TRDAR_CD': '상권_코드',
    'TRDAR_CD_NM': '상권_명',
    'XCNTS_VALUE': 'X좌표',
    'YDNTS_VALUE': 'Y좌표',
    'SIGNGU_CD': '자치구_코드',
    'SIGNGU_CD_NM': '자치구_명',
    'ADSTRD_CD': '행정동_코드',
    'ADSTRD_CD_NM': '행정동_코드_명',
    'RELM_AR': '영역_면적',
}


# ============================================================
# 행정동별 상권유형 개수 집계 함수
# ============================================================
def aggregate_by_dong(df):
    """
    행정동별 상권유형 개수 집계
    - 골목상권, 발달상권, 전통시장, 관광특구 각각 몇 개인지
    """
    print(f"\n{'='*60}")
    print("📊 행정동별 상권유형 개수 집계")
    print(f"{'='*60}")
    
    # 상권유형별 개수 피벗
    pivot = df.pivot_table(
        index=['행정동_코드', '행정동_코드_명', '자치구_명'],
        columns='상권구분_명',
        values='상권_코드',
        aggfunc='count',
        fill_value=0
    ).reset_index()
    
    # 컬럼명 정리
    pivot.columns.name = None
    
    # 전체 상권 수 추가
    상권유형_컬럼 = [col for col in pivot.columns if col not in ['행정동_코드', '행정동_코드_명', '자치구_명']]
    pivot['전체_상권수'] = pivot[상권유형_컬럼].sum(axis=1)
    
    print(f"집계 완료: {len(pivot)}개 행정동")
    print(f"\n상권유형 컬럼: {상권유형_컬럼}")
    
    return pivot, 상권유형_컬럼


# ============================================================
# EDA 함수
# ============================================================
def run_eda(df_raw, df_agg, 상권유형_컬럼):
    """
    탐색적 데이터 분석 (EDA) - 상권영역
    - 원본 데이터(df_raw): 상권별 데이터
    - 집계 데이터(df_agg): 행정동별 상권유형 개수
    """
    print(f"\n{'='*60}")
    print("📊 EDA (탐색적 데이터 분석) 시작 - 상권영역")
    print(f"{'='*60}")
    sys.stdout.flush()
    
    # =========================================================
    # 1단계: 원본 데이터 훑어보기
    # =========================================================
    print(f"\n{'='*60}")
    print("📌 1단계: 원본 데이터 훑어보기 (상권별)")
    print(f"{'='*60}")
    
    print("\n[1-1] 데이터 크기")
    print("-" * 40)
    print(f"전체 상권 수: {len(df_raw):,}개")
    print(f"행정동 수: {df_raw['행정동_코드_명'].nunique()}개")
    print(f"자치구 수: {df_raw['자치구_명'].nunique()}개")
    
    print("\n[1-2] 상권유형별 개수")
    print("-" * 40)
    상권유형_분포 = df_raw['상권구분_명'].value_counts()
    for 유형, 개수 in 상권유형_분포.items():
        print(f"  - {유형}: {개수:,}개 ({개수/len(df_raw)*100:.1f}%)")
    
    print("\n[1-3] 자치구별 상권 수 (상위 10개)")
    print("-" * 40)
    자치구별 = df_raw['자치구_명'].value_counts().head(10)
    for 구, 개수 in 자치구별.items():
        print(f"  - {구}: {개수:,}개")
    
    print("\n[1-4] 상위 5개 행 미리보기")
    print("-" * 40)
    print(df_raw.head())
    
    print("\n[1-5] 결측치 확인")
    print("-" * 40)
    missing = df_raw.isnull().sum()
    missing_with_values = missing[missing > 0]
    if len(missing_with_values) > 0:
        print(missing_with_values)
    else:
        print("✅ 결측치 없음!")
    
    sys.stdout.flush()
    
    # =========================================================
    # 2단계: 집계 데이터 확인 (행정동별)
    # =========================================================
    print(f"\n{'='*60}")
    print("📌 2단계: 집계 데이터 확인 (행정동별 상권유형 개수)")
    print(f"{'='*60}")
    
    print("\n[2-1] 데이터 크기")
    print("-" * 40)
    print(f"행정동 수: {len(df_agg):,}개")
    print(f"컬럼: {df_agg.columns.tolist()}")
    
    print("\n[2-2] 기초 통계량 (상권유형별 개수)")
    print("-" * 40)
    for col in 상권유형_컬럼:
        stats = df_agg[col].describe()
        print(f"\n  [{col}]")
        print(f"    평균: {stats['mean']:.1f}개")
        print(f"    최소: {stats['min']:.0f}개")
        print(f"    최대: {stats['max']:.0f}개")
        print(f"    중앙값: {stats['50%']:.0f}개")
    
    print("\n[2-3] 상위 10개 행 미리보기")
    print("-" * 40)
    print(df_agg.head(10))
    
    sys.stdout.flush()
    
    # =========================================================
    # 3단계: 그래프로 보기
    # =========================================================
    print(f"\n{'='*60}")
    print("📌 3단계: 그래프로 보기")
    print(f"{'='*60}")
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 3-1. 상권유형별 분포 (파이차트)
    colors = ['#ff9999', '#66b3ff', '#99ff99', '#ffcc99']
    axes[0, 0].pie(상권유형_분포.values, labels=상권유형_분포.index, autopct='%1.1f%%', colors=colors)
    axes[0, 0].set_title('상권유형별 분포', fontsize=12)
    
    # 3-2. 자치구별 상권 수 (막대그래프)
    자치구별_전체 = df_raw['자치구_명'].value_counts()
    axes[0, 1].barh(자치구별_전체.index[::-1], 자치구별_전체.values[::-1], color='steelblue')
    axes[0, 1].set_title('자치구별 상권 수', fontsize=12)
    axes[0, 1].set_xlabel('상권 수')
    
    # 3-3. 행정동별 전체 상권 수 분포 (히스토그램)
    axes[1, 0].hist(df_agg['전체_상권수'], bins=30, edgecolor='black', alpha=0.7)
    axes[1, 0].set_title('행정동별 전체 상권 수 분포', fontsize=12)
    axes[1, 0].set_xlabel('상권 수')
    axes[1, 0].set_ylabel('행정동 수')
    
    # 3-4. 상권유형별 행정동당 평균 개수 (막대그래프)
    평균_개수 = df_agg[상권유형_컬럼].mean()
    axes[1, 1].bar(평균_개수.index, 평균_개수.values, color='coral', edgecolor='black')
    axes[1, 1].set_title('행정동당 평균 상권 수 (유형별)', fontsize=12)
    axes[1, 1].set_xlabel('상권유형')
    axes[1, 1].set_ylabel('평균 개수')
    axes[1, 1].tick_params(axis='x', rotation=15)
    
    plt.tight_layout()
    plt.savefig('eda_상권영역_시각화.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("✅ 시각화 저장 완료: eda_상권영역_시각화.png")
    
    sys.stdout.flush()
    
    # =========================================================
    # 4단계: 이상한 점 발견하기
    # =========================================================
    print(f"\n{'='*60}")
    print("📌 4단계: 이상한 점 발견하기")
    print(f"{'='*60}")
    
    # ---------------------------------------------------------
    # 4-1. 상권이 0개인 유형 확인
    # ---------------------------------------------------------
    print("\n[4-1] 상권이 없는 행정동 확인")
    print("-" * 40)
    
    zero_all = df_agg[df_agg['전체_상권수'] == 0]
    print(f"전체 상권 0개인 행정동: {len(zero_all)}개")
    
    if len(zero_all) > 0:
        print("\n⚠️ 상권이 하나도 없는 행정동:")
        print(zero_all[['행정동_코드_명', '자치구_명', '전체_상권수']].to_string(index=False))
    
    for col in 상권유형_컬럼:
        zero_count = (df_agg[col] == 0).sum()
        print(f"  - {col} 0개인 행정동: {zero_count}개 ({zero_count/len(df_agg)*100:.1f}%)")
    
    # ---------------------------------------------------------
    # 4-2. 이상치 확인 (전체 상권 수 기준)
    # ---------------------------------------------------------
    print("\n[4-2] 이상치 확인 (전체_상권수 - IQR 방식)")
    print("-" * 40)
    
    q1 = df_agg['전체_상권수'].quantile(0.25)
    q3 = df_agg['전체_상권수'].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    
    print(f"\n📐 IQR 이상치 판별 기준:")
    print(f"  - Q1 (25%): {q1:.0f}개")
    print(f"  - Q3 (75%): {q3:.0f}개")
    print(f"  - IQR: {iqr:.0f}개")
    print(f"  - 하한 경계: {lower:.0f}개")
    print(f"  - 상한 경계: {upper:.0f}개")
    
    # 하한 이상치
    outliers_low = df_agg[df_agg['전체_상권수'] < lower]
    print(f"\n[하한 이상치] 상권 수 < {lower:.0f}개: {len(outliers_low)}건")
    
    if len(outliers_low) > 0:
        outliers_low_save = outliers_low[['행정동_코드_명', '자치구_명', '전체_상권수'] + 상권유형_컬럼].copy()
        outliers_low_save = outliers_low_save.sort_values('전체_상권수')
        outliers_low_save.to_csv('eda_상권_하한이상치.csv', index=False, encoding='utf-8-sig')
        print(f"✅ CSV 저장 완료: eda_상권_하한이상치.csv ({len(outliers_low)}건)")
        
        print("\n⚠️ 하한 이상치 TOP 10 (상권 가장 적은 행정동):")
        for idx, row in outliers_low.nsmallest(10, '전체_상권수').iterrows():
            print(f"  - {row['행정동_코드_명']} ({row['자치구_명']}): {row['전체_상권수']:.0f}개")
    
    # 상한 이상치
    outliers_high = df_agg[df_agg['전체_상권수'] > upper]
    print(f"\n[상한 이상치] 상권 수 > {upper:.0f}개: {len(outliers_high)}건")
    
    if len(outliers_high) > 0:
        outliers_high_save = outliers_high[['행정동_코드_명', '자치구_명', '전체_상권수'] + 상권유형_컬럼].copy()
        outliers_high_save = outliers_high_save.sort_values('전체_상권수', ascending=False)
        outliers_high_save.to_csv('eda_상권_상한이상치.csv', index=False, encoding='utf-8-sig')
        print(f"✅ CSV 저장 완료: eda_상권_상한이상치.csv ({len(outliers_high)}건)")
        
        print("\n⚠️ 상한 이상치 TOP 10 (상권 가장 많은 행정동):")
        for idx, row in outliers_high.nlargest(10, '전체_상권수').iterrows():
            print(f"  - {row['행정동_코드_명']} ({row['자치구_명']}): {row['전체_상권수']:.0f}개")
            for 유형 in 상권유형_컬럼:
                if row[유형] > 0:
                    print(f"      └ {유형}: {row[유형]:.0f}개")
    
    total_outliers = len(outliers_low) + len(outliers_high)
    print(f"\n📊 이상치 요약:")
    print(f"  - 하한 이상치: {len(outliers_low)}건")
    print(f"  - 상한 이상치: {len(outliers_high)}건")
    print(f"  - 전체 이상치: {total_outliers}건 ({total_outliers/len(df_agg)*100:.1f}%)")
    
    # ---------------------------------------------------------
    # 4-3. 중복 확인
    # ---------------------------------------------------------
    print("\n[4-3] 중복 데이터 확인")
    print("-" * 40)
    
    dup_raw = df_raw.duplicated(subset=['상권_코드']).sum()
    print(f"원본 데이터 - 상권코드 중복: {dup_raw}건")
    
    dup_agg = df_agg.duplicated(subset=['행정동_코드']).sum()
    print(f"집계 데이터 - 행정동코드 중복: {dup_agg}건")
    
    sys.stdout.flush()
    
    # =========================================================
    # EDA 요약
    # =========================================================
    print(f"\n{'='*60}")
    print("📊 EDA 요약 - 상권영역")
    print(f"{'='*60}")
    print(f"전체 상권 수: {len(df_raw):,}개")
    print(f"행정동 수: {len(df_agg)}개")
    print(f"자치구 수: {df_raw['자치구_명'].nunique()}개")
    print(f"\n상권유형별 분포:")
    for 유형, 개수 in 상권유형_분포.items():
        print(f"  - {유형}: {개수:,}개 ({개수/len(df_raw)*100:.1f}%)")
    
    issues = []
    if len(zero_all) > 0:
        issues.append(f"상권 0개 행정동: {len(zero_all)}개")
    if total_outliers > 0:
        issues.append(f"이상치: {total_outliers}건")
    if dup_raw > 0:
        issues.append(f"중복 상권: {dup_raw}건")
    
    if issues:
        print(f"\n⚠️ 발견된 이슈:")
        for issue in issues:
            print(f"  - {issue}")
    else:
        print("\n✅ 특별한 이슈 없음!")
    
    print(f"\n📁 저장된 CSV 파일 목록:")
    print("  - 서울시_상권영역_원본.csv (전체 상권 데이터)")
    print("  - 서울시_상권영역_행정동별.csv (행정동별 상권유형 개수)")
    if len(outliers_low) > 0:
        print("  - eda_상권_하한이상치.csv")
    if len(outliers_high) > 0:
        print("  - eda_상권_상한이상치.csv")
    
    print(f"\n{'='*60}")
    print("📊 EDA 완료!")
    print(f"{'='*60}")
    
    return df_agg


# ============================================================
# 메인 실행
# ============================================================
if __name__ == "__main__":
    print("🚀 프로그램 시작")
    print(f"현재 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    sys.stdout.flush()
    
    # 1. 데이터 수집
    raw_data = fetch_all_data()
    
    if raw_data:
        # 2. DataFrame 변환
        df_raw = pd.DataFrame(raw_data)
        
        # 3. 컬럼명 한글 변환
        existing_cols = {k: v for k, v in COLUMN_MAPPING.items() if k in df_raw.columns}
        df_raw = df_raw.rename(columns=existing_cols)
        
        # 4. 원본 데이터 저장
        df_raw.to_csv('서울시_상권영역_원본.csv', index=False, encoding='utf-8-sig')
        print(f"\n✅ 저장 완료: 서울시_상권영역_원본.csv ({len(df_raw)}건)")
        
        # 5. 행정동별 상권유형 개수 집계
        df_agg, 상권유형_컬럼 = aggregate_by_dong(df_raw)
        
        # 6. 집계 데이터 저장
        df_agg.to_csv('서울시_상권영역_행정동별.csv', index=False, encoding='utf-8-sig')
        print(f"✅ 저장 완료: 서울시_상권영역_행정동별.csv ({len(df_agg)}건)")
        
        # 7. EDA 실행
        df_agg = run_eda(df_raw, df_agg, 상권유형_컬럼)
        
    else:
        print("데이터 수집 실패")
    
    print(f"\n🏁 프로그램 종료: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")